In [2]:
#!pip3 install PyQt5

In [ ]:
%matplotlib inline
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
import cv2
import numpy as np

import matplotlib.pyplot as plt
from main import files, paths

@tf.function
def detect_fn(model, image):
    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)
    return detections

def get_masks(image_path):
    # Load pipeline config and build a detection model
    configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
    detection_model = model_builder.build(model_config=configs['model'], is_training=False)

    # Restore checkpoint
    ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
    ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-3')).expect_partial()

    category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

    img = cv2.imread(image_path)

    image_np = np.array(img)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(detection_model, input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}

    print("detections => ", detections.keys())
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    import PIL.Image as Image

    boxes = detections['detection_boxes']
    # get all boxes from an array
    max_boxes_to_draw = boxes.shape[0]
    # get scores to get a threshold
    scores = detections['detection_scores']
    # this is set as a default but feel free to adjust it to your needs
    min_score_thresh=.9
    # # iterate over all objects found
    coordinates = []
    for i in range(boxes.shape[0]):
        if scores[i] > min_score_thresh:
            class_id = int(detections['detection_classes'][i] + label_id_offset)

            coordinates.append({
                "box": boxes[i],
                "class_name": category_index[class_id]["name"],
                "score": scores[i]
            })

    color_by_class = {"upper_clothes": (255, 255, 0), "lower_clothes": (0, 255, 255)}

    image_pil = Image.fromarray(np.uint8(image_np)).convert('RGB')

    im_width, im_height = image_pil.size

    masks = []

    for coordinate in coordinates:
        class_name = coordinate["class_name"]

        box = tuple(coordinate["box"].tolist())
        ymin, xmin, ymax, xmax = box

        (left, right, top, bottom) = (int(xmin * im_width), int(xmax * im_width),
                                        int(ymin * im_height), int(ymax * im_height))

        start_point = (left, top)
        end_point = (right, bottom)
        
        cv2.rectangle(img, start_point, end_point, color=color_by_class[class_name], thickness=3)

        (w, h), _ = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 2, 5)
        #img = cv2.rectangle(img, (left, right - 20), (left + w, right), color_by_class[class_name], -1)
        img = cv2.putText(img, class_name, (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color_by_class[class_name], 3)

        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        mask = img.copy()
        mask[:] = 255


        num_channels = 1 if len(mask.shape) == 2 else mask.shape[2]
        cv2.rectangle(mask, (left, top), (right, bottom), color=(0,) * num_channels, thickness=-1)

        masks.append({"name": class_name, "mask": mask})

    plt.imshow(mask)
    plt.show()

    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()


# TODO: pass image from command line arg
IMAGE_NAME = "2.jpg"
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'final_test', IMAGE_NAME)

get_masks(IMAGE_PATH)
# viz_utils.visualize_boxes_and_labels_on_image_array(
#             image_np_with_detections,
#             detections['detection_boxes'],
#             detections['detection_classes']+label_id_offset,
#             detections['detection_scores'],
#             category_index,
#             use_normalized_coordinates=True,
#             max_boxes_to_draw=5,
#             min_score_thresh=.8,
#             agnostic_mode=False)
# plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
# plt.show()

# plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
# plt.show()